In [3]:
import sys
import os
import os.path as op
import copy
import re
import zipfile
import json
import html
import htmltag
import bs4

HFS = os.environ['HFS']

sys.path.append(op.join(HFS, 'houdini/python2.7libs'))
from bookish.wikipages import parse_string as parse_wikipage

In [ ]:
# Open vex.zip containing various helpcards including functions.
# Parse Wiki files into JSON using Bookish parser.
functions = {}

with zipfile.ZipFile(op.join(utils.HFS, 'houdini/help/vex.zip')) as z:
    for path in z.namelist():
        if op.dirname(path) == 'functions':
            with z.open(path) as f:
                name = path[10:-4]
                markup = f.read().decode()
                functions[name] = parse_wikipage(markup)

In [ ]:
parsed = parse_sigs(set(all_usages(functions)))

# Override documentation signatures.
with open('functions.vfl') as f:
    overrides = parse_sigs(f.read().split('\n'))
    
for f, sigs in overrides.items():
    # Function not exists.
    if f not in parsed:
        parsed[f] = sigs
        continue

    # Function exists. Search for unique signatures.
    for over_s in sigs['sigs']:
        for s in copy.deepcopy(parsed[f]['sigs']):
            # Non-unique signature found. Override argument names.
            if s['args'] == over_s['args']:
                parsed[f]['sigs'].remove(s)
                parsed[f]['sigs'].insert(0, over_s)
                break  # Suppress for-else clause.

        # Non-unique signatures not found (= the signature is unique).
        else:
            parsed[f]['sigs'].insert(0, over_s)
            
sum(len(parsed[f]['sigs']) for f in parsed)

In [ ]:
# Inspect Bookish types.
def types(node):
    '''Recursively extract all 'type' values from Bookish tree.'''
    if type(node) is dict:
        yield node['type']

        for k in node:
            if type(node[k]) is list:
                for i in node[k]:
                    for j in types(i):
                        yield j
                        
def all_types(functions):
    for f in functions:
        for t in types(functions[f]):
            yield t
    

bookish_types = set(all_types(functions))
print(' '.join(sorted(bookish_types)))
print('Total:', len(bookish_types))

In [ ]:
from htmltag import code, div, em, h1, h2, p, span, strong


class Wrapper:
    '''Convert Bookish JSON structure to html.'''

    def __init__(self):
        # Trivial type handlers.
        self.arg = lambda n: div(self(n['text']), _class='arg')
        self.bullet = lambda n: div(self(n['text']), _class='bullet')
        self.code = lambda n: code(self(n['text']))
        self.dt = lambda n: div(self(n['text']), _class='dt')
        self.em = lambda n: em(self(n['text']))
        self.h = lambda n: h2(self(n['text']))
        self.ord = self.bullet
        self.para = lambda n: p(self(n['text']))
        self.pre = lambda n: div(self.code(n), _class='pre')
        self.prop = lambda n: div(self(n['value']), _class='prop %s' % n['name'])
        self.root = lambda n: div(self(n['body']), _class='helpcard')
        self.strong = lambda n: strong(self(n['text']))
        self.summary = lambda n: p(self(n['text']), _class='summary')
        self.title = lambda n: h1(self(n['text']))
        self.ui = lambda n: span(self(n['text']), _class='ui')
        self.usage = lambda n: div(self(n['text']), _class='usage')
        self.var = lambda n: code(self(n['text']), _class='var')
        self.xml = lambda n: span(self(n['text']), _class='xmltag %s' % n['tag'])
        self.examples_section = lambda n: div('', _class='examples_section', _id=n['id'])
        self.parameters_section = lambda n: div('', _class='parameters_section', _id=n['id'])
        self.related_section = lambda n: div('', _class='related_section', _id=n['id'])
        self.return_section = lambda n: div(self(n['text']), _class='return_section', _id=n['id'])
        self.syntax_section = lambda n: div('', _class='syntax_section', _id=n['id'])
        self.box = lambda n: ''
        self.note = lambda n: ''
        self.pxml = lambda n: ''
        self.returns = lambda n: ''
        self.tip = lambda n: ''
        self.warning = lambda n: ''

    def link(self, n):
        if n['scheme'] == 'Include':
            # Do not handle anything for now.
            return ''
    
        base = {
            'Hom': 'https://www.sidefx.com/docs/houdini/hom/hou/',
            'Hprop': 'https://www.sidefx.com/docs/houdini/props/mantra#',
            'Image': 'https://www.sidefx.com/docs/houdini',
            'Node': 'https://www.sidefx.com/docs/houdini/nodes/',
            'Vex': 'https://www.sidefx.com/docs/houdini/vex/functions/',
            'Wp': 'https://en.wikipedia.org/wiki/',
            None: 'https://www.sidefx.com/docs/houdini',
        }[n['scheme']]

        rest = n['value']
        if n['scheme'] == 'Hom':
            rest = rest.rsplit('.')[-1]

        content = n['value'] if not n['text'] else n['text']
        return htmltag.a(self(content), href=base+rest)

    def __call__(self, node):
        if type(node) is dict:
            return getattr(self, node['type'])(node)
        elif type(node) is list:
            return htmltag.HTML(''.join(self(i) for i in node))
        return html.escape(node)


wrapper = Wrapper()

for f in functions:
    wrapper(functions[f])

# function = 'chsetattr'
# with open('helpcards.json') as infile:
#     old_page = json.load(infile)['vex'][function]
#     with open('out/%s.html' % function, 'w', encoding='utf-8') as outfile:
#         outfile.write(old_page)
# with open('out/%s.json' % function, 'w') as f:
#     json.dump(functions[function], f, indent=4)
# soup = bs4.BeautifulSoup(wrapper(functions[function]), 'html.parser')
# print(soup.prettify())